In [13]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [25]:
df = pd.read_csv('goodreads_data.csv')

df['Genres'] = df['Genres'].apply(ast.literal_eval)

all_genres = set([genre for sublist in df['Genres'] for genre in sublist])

genre_columns = pd.DataFrame({genre: df['Genres'].apply(lambda x: 1 if genre in x else 0) for genre in all_genres})

df = pd.concat([df, genre_columns], axis=1)

print(df)

      Unnamed: 0                                               Book  \
0              0                              To Kill a Mockingbird   
1              1  Harry Potter and the Philosopher’s Stone (Harr...   
2              2                                Pride and Prejudice   
3              3                          The Diary of a Young Girl   
4              4                                        Animal Farm   
...          ...                                                ...   
9995        9995                    Breeders (Breeders Trilogy, #1)   
9996        9996                                             Dynamo   
9997        9997                              The Republic of Trees   
9998        9998                     Waking Up (Healing Hearts, #1)   
9999        9999                 Bits and Pieces: Tales and Sonnets   

                 Author                                        Description  \
0            Harper Lee  The unforgettable novel of a childhood in a 

In [31]:
def convert_to_list(x):
    if isinstance(x, str):
        # If it's a string, attempt to parse it as a list
        return ast.literal_eval(x)
    elif isinstance(x, list):
        # If it's already a list, return it as is
        return x
    else:
        return []  # Handle any unexpected formats

df['Genres'] = df['Genres'].apply(convert_to_list)

unique_genres = set([genre for sublist in df['Genres'] for genre in sublist])

unique_genre_list = sorted(list(unique_genres))

print(unique_genre_list)

['12th Century', '15th Century', '16th Century', '17th Century', '18th Century', '19th Century', '20th Century', '21st Century', 'Abuse', 'Academia', 'Academic', 'Action', 'Activism', 'Adhd', 'Adoption', 'Adult', 'Adult Fiction', 'Adventure', 'Africa', 'African American', 'African American Literature', 'African Literature', 'Agriculture', 'Alchemy', 'Algeria', 'Aliens', 'Alternate History', 'Alternate Universe', 'Amazon', 'American', 'American Civil War', 'American History', 'American Revolution', 'American Revolutionary War', 'Americana', 'Amish', 'Anarchism', 'Ancient', 'Ancient History', 'Angels', 'Animal Fiction', 'Animals', 'Anthologies', 'Anthropology', 'Anti Racist', 'Apocalyptic', 'Archaeology', 'Architecture', 'Art', 'Art Design', 'Art History', 'Arthurian', 'Artificial Intelligence', 'Asexual', 'Asia', 'Asian Literature', 'Astrology', 'Astronomy', 'Atheism', 'Audiobook', 'Australia', 'Autistic Spectrum Disorder', 'Autobiography', 'Aviation', 'BDSM', 'Bande Dessinée', 'Banned 

In [75]:
def recommend(selected_genres):
    user_profile = {genre: 1 if genre in selected_genres else 0 for genre in all_genres}
    user_profile_vector = pd.DataFrame([user_profile])
    
    book_genre_vectors = df[list(all_genres)]
    
    similarity_scores = cosine_similarity(user_profile_vector, book_genre_vectors)
    
    df['matching_genres'] = df['Genres'].apply(lambda x: len(set(x) & set(selected_genres)))
    
    df['similarity'] = similarity_scores[0] + df['matching_genres']
    
    df_sorted = df.sort_values(by='similarity', ascending=False)
    
    df_filtered = df_sorted[df_sorted['matching_genres'] > 0]
    
    N = 3
    top_recommendations = df_filtered[['Book', 'Genres', 'similarity']].head(N)
    
    print(top_recommendations)

In [73]:
user_genres = ['Academic', 'Fiction', 'Dark']
recommend(user_genres)

                                            Book  \
2546                                    Not Okay   
1403       A Little Pinprick (Rainey Paxton, #1)   
2625  A Disturbed Girl's Guide to Curing Boredom   

                                 Genres  similarity  
2546      [Fiction, Dark, Contemporary]    2.666667  
1403             [Fiction, Abuse, Dark]    2.666667  
2625  [Fiction, Thriller, Horror, Dark]    2.577350  
